# Downloading Stock Price Data

This notebook downloads historical stock data using the `yfinance` API and prepares both daily and monthly datasets for analysis.


In [25]:
import yfinance as yf
import pandas as pd

# Select tickers 
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'UNH', 'HD']

# Download close prices
prices = yf.download(tickers, start="2014-01-01", end="2024-12-31", auto_adjust=True)['Close']
daily_returns = prices.pct_change().dropna()

# Save raw prices and returns
prices.to_csv('../data/raw/prices.csv')
daily_returns.to_csv('../data/raw/returns_daily.csv')

# Resample to monthly
monthly_prices = prices.resample('ME').last()
monthly_returns = monthly_prices.pct_change().dropna()
monthly_prices.to_csv('../data/processed/prices_monthly.csv')
monthly_returns.to_csv('../data/processed/returns_monthly.csv')

[*********************100%***********************]  10 of 10 completed


In [24]:
from pathlib import Path

FF5_CSV = Path('../data/raw/F-F_Research_Data_5_Factors_2x3.csv')
if not FF5_CSV.exists():
    raise FileNotFoundError(
        f"Couldn't find {FF5_CSV}. Download the CSV named 'F-F_Research_Data_5_Factors_2x3.csv' (Monthly) from the Kenneth French Data Library" 
    )

# read and clean
raw = pd.read_csv(FF5_CSV, skiprows=3)
raw = raw.rename(columns={raw.columns[0]: "Date"})
raw = raw[raw["Date"].astype(str).str.match(r"^\d{6}$", na=False)]
raw = raw.iloc[:, :7]

cols = ["Date", "Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"]
raw.columns = cols[:raw.shape[1]]

raw["Date"] = pd.to_datetime(raw["Date"], format="%Y%m")
raw.set_index("Date", inplace=True)
ff5 = raw.astype(float) / 100.0
ff5.index = ff5.index + pd.offsets.MonthEnd(0)

ff5.to_csv("../data/processed/ff5_monthly_clean.csv")
ff5.head()


,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
1963-07-31,-0.0039,-0.0048,-0.0081,0.0064,-0.0115,0.0027
1963-08-31,0.0508,-0.0080,0.0170,0.0040,-0.0038,0.0025
1963-09-30,-0.0157,-0.0043,0.0000,-0.0078,0.0015,0.0027
1963-10-31,0.0254,-0.0134,-0.0004,0.0279,-0.0225,0.0029
1963-11-30,-0.0086,-0.0085,0.0173,-0.0043,0.0227,0.0027


In [23]:
# stack returns and merge with FF5 on date
panel = monthly_returns.stack().rename("ret").to_frame()
panel.index.set_names(["date", "ticker"], inplace=True)

ff5 = ff5.copy()
ff5.index.name = "date"
panel = (
    panel.reset_index()
         .merge(ff5.reset_index(), how="inner", on="date")
         .set_index(["date", "ticker"])
         .sort_index()
)

# compute excess returns and save
panel["excess_ret"] = panel["ret"] - panel["RF"]
panel.to_csv("../data/processed/panel_monthly_with_ff5.csv")
panel.tail()


ret  Mkt-RF     SMB   HML    RMW     CMA      RF  \
date       ticker                                                          
2024-12-31 META    0.030291 -0.0315 -0.0383 -0.03  0.019 -0.0121  0.0037   
           MSFT    0.003235 -0.0315 -0.0383 -0.03  0.019 -0.0121  0.0037   
           NVDA   -0.005429 -0.0315 -0.0383 -0.03  0.019 -0.0121  0.0037   
           TSLA    0.209323 -0.0315 -0.0383 -0.03  0.019 -0.0121  0.0037   
           UNH    -0.164622 -0.0315 -0.0383 -0.03  0.019 -0.0121  0.0037   

                   excess_ret  
date       ticker              
2024-12-31 META      0.026591  
           MSFT     -0.000465  
           NVDA     -0.009129  
           TSLA      0.205623  
           UNH      -0.168322